The World Band > [Life expectancy at birth, total](https://data.worldbank.org/indicator/SP.DYN.LE00.IN)<br />
The World Band > [Life expectancy at birth, male](https://data.worldbank.org/indicator/SP.DYN.LE00.MA.IN)<br />
The World Band > [Life expectancy at birth, female](https://data.worldbank.org/indicator/SP.DYN.LE00.FE.IN)<br />
[World Development Indicators > Life expectancy at birth](https://databank.worldbank.org/reports.aspx?source=2&series=SP.DYN.LE00.IN&country=) *(databank.worldbank.org)*<br />
[OECD stat](https://stats.oecd.org/index.aspx?queryid=30114)

"[Список стран по ожидаемой продолжительности жизни](https://ru.wikipedia.org/wiki/Список_стран_по_ожидаемой_продолжительности_жизни)"<br />
"[Продолжительность жизни в субъектах Российской Федерации](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_субъектах_Российской_Федерации)"<br />
[Таблица подбора цветов](http://mal-bioit.ru/survey-web-colors)<br />
[Список государств и зависимых территорий по населению](https://ru.wikipedia.org/wiki/Список_государств_и_зависимых_территорий_по_населению)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import re

In [2]:
pd.options.display.max_rows = 300

DESTINATION_OUTPUT = ''  # to where table code should be placed: 'file', 'here' or just print 'Done'

In [3]:
def output_table_code(st, file_name='', destination=''):
    if not destination:
        destination = DESTINATION_OUTPUT
        
    if destination == 'file':
        with open('output/'+file_name, 'w', encoding="utf-8") as fh:
            fh.write(st)
        print('Data has written to file')
    elif destination == 'here':
        print(st)
    else:
        print('done')

In [4]:
# create code for placing info in Wikipedia
def create_table(df, lang='en'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=2):  # change_value
        return 'style="background:#fffae0;"|—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"| −{x:0.{prec}f}'  # normal sorting is required '−' before 0.0 - bug in wiki
    
    def chval_bold(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    if lang=='ru':
        file_header='wbg_stats_header_2021_ru.txt'
        ptn_1 = 'флагификация'
        ptn_2 = 'флаг'
        prettify_name = {
            'European Union': '[[Европейский союз]]',
            'Central Europe and the Baltics': '[[Центральная Европа]] и [[Прибалтика]]',
            'Europe & Central Asia': '[[Европа]] и [[Центральная Азия]]',
            'South Asia': '[[Южная Азия]]',
            'East Asia & Pacific': '[[Восточная Азия]] и [[Австралия и Океания|Океания]]',
            'Middle East & North Africa': '[[Страны MENA|Ближний Восток и Северная Африка]]',
            'Africa Eastern and Southern': '[[Восточная Африка|Восточная]] и [[Южная Африка|Южная]] Африка',
            'Sub-Saharan Africa': '[[Чёрная Африка|Африка к югу от Сахары]]',
            'Africa Western and Central': '[[Западная Африка|Западная]] и [[Центральная Африка|Центральная]] Африка',
            'North America': '[[Северная Америка]]',
            'Latin America & Caribbean': '[[Латинская Америка]] и [[Антильские острова|Карибы]]',
            'Arab World': 'Арабский мир',
            'World': 'Мир',
            'Caribbean small states': '[[Список стран Карибского бассейна|Малые страны Карибского бассейна]]',
            'Pacific island small states': 'Малые страны Тихого океана',
            'Small states': 'Малые страны',
            'Other small states': 'Другие малые страны'
        }
    else:
        file_header='wbg_stats_header_2021_en.txt'
        ptn_1 = 'flaglist'
        ptn_2 = 'flagicon'
        prettify_name = {
            'European Union': '[[European Union]]',
            'Central Europe and the Baltics': '[[Central Europe]] & the [[Baltic states|Baltics]]',
            'Europe & Central Asia': '[[Europe]] & [[Central Asia]]',
            'South Asia': '[[South Asia]]',
            'East Asia & Pacific': '[[Asia-Pacific|East Asia & Pacific]]',
            'Middle East & North Africa': '[[MENA|Middle East & North Africa]]',
            'Africa Eastern and Southern': 'Africa [[East Africa|Eastern]] & [[Southern Africa|Southern]]',
            'Sub-Saharan Africa': '[[Sub-Saharan Africa]]',
            'Africa Western and Central': 'Africa [[West Africa|Western]] & [[Central Africa|Central]]',
            'North America': '[[North America]]',
            'Latin America & Caribbean': '[[Latin America and the Caribbean|Latin America & Caribbean]]'
        }
        
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        st_header = fh.read()
        
    st_header = st_header.strip()
    st = ''
        
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name in ['World',
                        'European Union', 'Central Europe and the Baltics',
                        'Europe & Central Asia',
                        'South Asia',
                        'East Asia & Pacific',
                        'Middle East & North Africa',
                        'Africa Eastern and Southern', 'Sub-Saharan Africa', 'Africa Western and Central',
                        'North America', 'Latin America & Caribbean']:
            pass
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|align=center| \'\'\'{prettify_name.get(ser.name, ser.name)}\'\'\' ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#eaf3ff;"| \'\'\'{if_value(ser.male, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#fee7f6;"| \'\'\'{if_value(ser.female, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#fff8dc;"| \'\'\'{if_value(ser.gender_gap, 2)}\'\'\' ' + \
                  f'||style="text-align:center; border-left-width:2px; padding-left:1em;"| \'\'\'{if_value(ser["2014_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2019_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2020_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2014→2021"], 2)}'
        elif ser.name in ['Caribbean small states',
                          'Pacific island small states',
                          'Small states',
                          'Other small states']:
            st += '\n' + '|-\n' + \
                  f'|style="text-align:left; padding-left: 35px;"|{prettify_name.get(ser.name, ser.name)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||style="text-align:center; background:#eaf3ff;"| {if_value(ser.male, 2)} ' + \
                  f'||style="text-align:center; background:#fee7f6;"| {if_value(ser.female, 2)} ' + \
                  f'||style="text-align:center; background:#fff8dc;"| {if_value(ser.gender_gap, 2)} ' + \
                  f'||style="text-align:center; border-left-width:2px; padding-left:1em;"| {if_value(ser["2014_t"], 2)} ' + \
                  f'||{chval(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2019_t"], 2)} ' + \
                  f'||{chval(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2020_t"], 2)} ' + \
                  f'||{chval(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||{chval(ser["2014→2021"], 2)}'
        else:
            st += '\n' + '|-\n' + \
                  f'|align=left|{{{{{ptn_1}|{ser.name}}}}} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||style="text-align:center; background:#eaf3ff;"| {if_value(ser.male, 2)} ' + \
                  f'||style="text-align:center; background:#fee7f6;"| {if_value(ser.female, 2)} ' + \
                  f'||style="text-align:center; background:#fff8dc;"| {if_value(ser.gender_gap, 2)} ' + \
                  f'||style="text-align:center; border-left-width:2px; padding-left:1em;"| {if_value(ser["2014_t"], 2)} ' + \
                  f'||{chval(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2019_t"], 2)} ' + \
                  f'||{chval(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2020_t"], 2)} ' + \
                  f'||{chval(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||{chval(ser["2014→2021"], 2)}'
    st += '\n|}'
    
    # WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    st = st_header + st
    
    return st

In [5]:
def extract_data_for_year(df, year='2019'):
    ls_dropping = [
        'Hong Kong SAR, China',     # administrative region of China, population 7,291,000
        'Macao SAR, China',         # administrative region of China, population 682,000
        'Kosovo',                   # partially recognised state, population 1,806,000
        # 'West Bank and Gaza',        # Palestine, partially recognised state, population 5,227,000
        # 'New Caledonia',             # French Overseas Territory in Oceania, population 271,000
        # 'Channel Islands',           # archipelago in Europe, consisting of two Crown Dependencies, population 172,000
        # 'Curacao',                   #  constituent country of the Netherlands in S.America, population 149,000, little info about LE
        # 'Virgin Islands (U.S.)',     # unincorporated and organized territory of USA, population 87,000
        # 'Isle of Man',               # self-governing Crown Dependency, Europe, population 84,000
        # 'Dominica',                  # micro-state in N.America, population 72,400
        # 'Marshall Islands',          # micro-state in Oceania, population 62,000
        # 'Faroe Islands',             # autonomous territory of Denmark in Europe, population 54,000
        # 'St. Kitts and Nevis',       # micro-state in N.America, population 47,600
        # 'Turks and Caicos Islands',  # British Overseas Territory in N.America, population 44,000
        # 'Sint Maarten (Dutch part)', # constituent country of the Netherlands in N.America, population 41,500
        # 'Gibraltar',                 # British Overseas Territory in Europe, population 34,000
        # 'St. Martin (French part)',  # French Overseas Territory in N.America, population 32,500
        # 'British Virgin Islands',    # British Overseas Territory in N.America, population 30,000
        # 'Palau',           #  mini-state in Oceania, population 22,000, little info about LE
        # 'Tuvalu',                    # island micro-state in Oceania, population 12,000
        # 'Nauru'                      # micro-state in Oceania, population 11,000
        'Euro area',
        'IBRD only',
        'IDA & IBRD total',
        'IDA total',
        'IDA blend',
        'IDA only',
        'OECD members',
        'Small states',
        'Other small states',
        'Europe & Central Asia (excluding high income)',
        'East Asia & Pacific (excluding high income)',
        'Latin America & Caribbean (excluding high income)',
        'Middle East & North Africa (excluding high income)',
        'Sub-Saharan Africa (excluding high income)',
        'East Asia & Pacific (IDA & IBRD countries)',
        'Europe & Central Asia (IDA & IBRD countries)',
        'Middle East & North Africa (IDA & IBRD countries)',
        'Sub-Saharan Africa (IDA & IBRD countries)',           
        'Latin America & the Caribbean (IDA & IBRD countries)',
        'South Asia (IDA & IBRD)',
        'Heavily indebted poor countries (HIPC)',
        'Fragile and conflict affected situations',
        'Least developed countries: UN classification',
        'Early-demographic dividend',
        'Late-demographic dividend',
        'Pre-demographic dividend',
        'Post-demographic dividend',
        'Low income',
        'Lower middle income',
        'Low & middle income',
        'Middle income',
        'Upper middle income',
        'High income'
    ]
    
    ser = df.drop(ls_dropping) \
            [str(year)] \
            .dropna() \
            .rename(index = {'United States':'USA',
                             'Russian Federation': 'Russia',
                             'Korea, Rep.': 'South Korea',
                             'Egypt, Arab Rep.': 'Egypt',
                             'Congo, Dem. Rep.': 'DR Congo',   #'Congo, Democratic Republic of',
                             'Congo, Rep.': 'Congo, Rep.', #'Republic of the Congo', #'Congo Republic',  #'Congo, Republic of',
                             'Yemen, Rep.': 'Yemen',
                             'Micronesia, Fed. Sts.': 'Micronesia',
                             'Gambia, The': 'Gambia',
                             'Iran, Islamic Rep.': 'Iran',
                             "Korea, Dem. People's Rep.": 'North Korea',
                             'Syrian Arab Republic': 'Syria',
                             'Venezuela, RB': 'Venezuela',
                             'Kyrgyz Republic': 'Kyrgyzstan',
                             'Lao PDR': 'Laos',
                             'Czech Republic': 'Czechia',
                             'Slovak Republic': 'Slovakia',
                             'Bahamas, The': 'Bahamas',
                             'Brunei Darussalam': 'Brunei',
                             'Turkiye': 'Turkey',
                             'West Bank and Gaza': 'Palestine',
                             'Sint Maarten (Dutch part)': 'Sint Maarten',
                             'St. Martin (French part)': 'Collectivity of Saint Martin'
                            })
    
    return ser

In [6]:
df = pd.read_csv('data/World_Bank_Group-total.csv', skiprows=4, index_col=0)

ser_2021_total = extract_data_for_year(df, 2021)
ser_2021_total.name = '2021_total'

ser_2020_total = extract_data_for_year(df, 2020)
ser_2020_total.name = '2020_total'

ser_2019_total = extract_data_for_year(df, 2019)
ser_2019_total.name = '2019_total'

ser_2014_total = extract_data_for_year(df, 2014)
ser_2014_total.name = '2014_total'

ser_2000_total = extract_data_for_year(df, 2000)
ser_2000_total.name = '2000_total'

del df

print(f"2021-total: {len(ser_2021_total)} records")
print(f"2020-total: {len(ser_2020_total)} records")
print(f"2019-total: {len(ser_2019_total)} records")
print(f"2014-total: {len(ser_2014_total)} records")
print(f"2000-total: {len(ser_2000_total)} records")

2021-total: 221 records
2020-total: 221 records
2019-total: 221 records
2014-total: 222 records
2000-total: 223 records


In [7]:
df = pd.read_csv('data/World_Bank_Group-male.csv', skiprows=4, index_col=0)

ser_2021_male = extract_data_for_year(df, 2021)
ser_2021_male.name = '2021_male'

del df

print(f"2021-male: {len(ser_2021_male)} records")

2021-male: 221 records


In [8]:
df = pd.read_csv('data/World_Bank_Group-female.csv', skiprows=4, index_col=0)

ser_2021_female = extract_data_for_year(df, 2021)
ser_2021_female.name = '2021_female'

del df

print(f"2021-female: {len(ser_2021_female)} records")

2021-female: 221 records


In [9]:
# concate series, removing records without data for last years
df = pd.concat([ser_2021_total, ser_2021_male, ser_2021_female, ser_2020_total, ser_2019_total], axis='columns', join='outer')  \
       .join([ser_2014_total, ser_2000_total], how='left')

df.index.name = ''

# change order of columls and rename them
df = df[['2021_total', '2021_male', '2021_female', '2000_total', '2014_total', '2019_total', '2020_total']]
df.rename(columns={'2021_total': 'total',
                   '2021_male': 'male',
                   '2021_female': 'female',
                   '2000_total': '2000_t',
                   '2014_total': '2014_t',
                   '2019_total': '2019_t',
                   '2020_total': '2020_t'}, inplace=True)

print(df.shape)
df.loc[['World', 'Russia']]

(221, 7)


,total,male,female,2000_t,2014_t,2019_t,2020_t
,,,,,,,
World,71.327718,68.886746,73.948295,67.700402,71.881296,72.979716,72.24385
Russia,69.361220,64.210000,74.770000,65.483659,70.743659,73.083902,71.33878


In [10]:
# calculate difference between some columns and insert result as new columns in dataframe
df.insert(loc=3, column='gender_gap', value=df['female']-df['male'])
df.insert(loc=5, column='2000→2014', value=df['2014_t']-df['2000_t'])
df.insert(loc=7, column='2014→2019', value=df['2019_t']-df['2014_t'])
df.insert(loc=9, column='2019→2020', value=df['2020_t']-df['2019_t'])
df.insert(loc=11, column='2020→2021', value=df['total']-df['2020_t'])
df.insert(loc=12, column='2014→2021', value=df['total']-df['2014_t'])

# sort DataFrame by total life expectancy
df.sort_values(by=['total', 'male', 'female'], ascending=False, inplace=True)

print(df.shape)
df.loc[['World', 'Russia']]

(221, 13)


,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
World,71.327718,68.886746,73.948295,5.061549,67.700402,4.180894,71.881296,1.098420,72.979716,-0.735866,72.24385,-0.916132,-0.553578
Russia,69.361220,64.210000,74.770000,10.560000,65.483659,5.260000,70.743659,2.340244,73.083902,-1.745122,71.33878,-1.977561,-1.382439


<br />
<br />
<br />

In [11]:
# ls_new = df.index.to_list()
# print(len(ls_new))

# ls_old = ['Japan', 'Singapore', 'South Korea', ...]

# print(len(ls_old))

In [12]:
# [country for country in ls_new if country not in ls_old]

In [13]:
# [country for country in ls_old if country not in ls_new]

<br />
<br />
<br />

In [14]:
ls_N_America = ['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica',
                'Cuba', 'Dominican Republic', 'El Salvador', 'Grenada', 'Guatemala',
                'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama',
                'St. Lucia', 'St. Vincent and the Grenadines',
                'Trinidad and Tobago', 'USA',
                'World',                
                'Bermuda', 'Puerto Rico', 'Virgin Islands (U.S.)', 'British Virgin Islands', 'Turks and Caicos Islands',
                'Caribbean small states', 'Sint Maarten', 'Collectivity of Saint Martin', 'Dominica', 'St. Kitts and Nevis',
                'Aruba', 'Greenland']

df_N_America = df.loc[ls_N_America]   \
                 .sort_values(by='total', ascending=False)
df_N_America

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Canada,82.596610,80.621000,84.671000,4.050000,79.166829,2.617561,81.784390,0.444390,82.228780,-0.558293,81.670488,0.926122,0.812220
Collectivity of Saint Martin,80.379000,77.254000,83.624000,6.370000,76.663000,3.303000,79.966000,0.017000,79.983000,0.166000,80.149000,0.230000,0.413000
Puerto Rico,80.162000,75.865000,84.522000,8.657000,75.977000,2.952000,78.929000,0.134000,79.063000,-1.022000,78.041000,2.121000,1.233000
Virgin Islands (U.S.),80.068293,76.800000,83.500000,6.700000,76.619512,2.248780,78.868293,0.800000,79.668293,0.151220,79.819512,0.248780,1.200000
Bermuda,79.280000,75.817000,82.708000,6.891000,78.858000,0.660000,79.518000,1.515000,81.033000,0.103000,81.136000,-1.856000,-0.238000
Antigua and Barbuda,78.497000,75.775000,80.941000,5.166000,74.606000,3.252000,77.858000,0.833000,78.691000,0.150000,78.841000,-0.344000,0.639000
Barbados,77.571000,75.629000,79.359000,3.730000,74.089000,2.387000,76.476000,0.781000,77.257000,0.136000,77.393000,0.178000,1.095000
Costa Rica,77.023000,74.418000,79.811000,5.393000,77.586000,1.188000,78.774000,0.653000,79.427000,-0.150000,79.277000,-2.254000,-1.751000
USA,76.329268,73.500000,79.300000,5.800000,76.636585,2.204878,78.841463,-0.053659,78.787805,-1.807317,76.980488,-0.651220,-2.512195


In [15]:
table_code_N_America = create_table(df_N_America)
output_table_code(table_code_N_America, 'Table code WBG -2021 -N_America -en.txt', destination='file')

Data has written to file


In [16]:
table_code_N_America = create_table(df_N_America, lang='ru')
output_table_code(table_code_N_America, 'Table code WBG -2021 -N_America -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [17]:
ls_S_America = ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador',
                'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela',
                'World']

df_S_America = df.loc[ls_S_America]  \
                 .sort_values(by='total', ascending=False)
df_S_America

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Chile,78.944000,76.472000,81.437000,4.965000,76.870000,2.603000,79.473000,0.85300,80.326000,-0.949000,79.37700,-0.433000,-0.529000
Uruguay,75.436000,71.654000,79.258000,7.604000,75.025000,2.341000,77.366000,0.14200,77.508000,0.922000,78.43000,-2.994000,-1.930000
Argentina,75.390000,72.182000,78.647000,6.465000,73.926000,2.829000,76.755000,0.52900,77.284000,-1.392000,75.89200,-0.502000,-1.365000
Ecuador,73.670000,70.260000,77.462000,7.202000,72.839000,3.783000,76.622000,0.67500,77.297000,-5.144000,72.15300,1.517000,-2.952000
Colombia,72.830000,69.404000,76.442000,7.038000,71.320000,4.723000,76.043000,0.70900,76.752000,-1.983000,74.76900,-1.939000,-3.213000
Brazil,72.750000,69.560000,76.009000,6.449000,69.737000,4.569000,74.306000,1.03200,75.338000,-1.329000,74.00900,-1.259000,-1.556000
Peru,72.377000,70.121000,74.746000,4.625000,70.424000,4.909000,75.333000,0.82300,76.156000,-2.491000,73.66500,-1.288000,-2.956000
World,71.327718,68.886746,73.948295,5.061549,67.700402,4.180894,71.881296,1.09842,72.979716,-0.735866,72.24385,-0.916132,-0.553578
Venezuela,70.554000,66.259000,75.207000,8.948000,72.478000,0.375000,72.853000,-0.69200,72.161000,-1.066000,71.09500,-0.541000,-2.299000


In [18]:
table_code_S_America = create_table(df_S_America)
output_table_code(table_code_S_America, 'Table code WBG -2021 -S_America -en.txt', destination='file')

Data has written to file


In [19]:
table_code_S_America = create_table(df_S_America, lang='ru')
output_table_code(table_code_S_America, 'Table code WBG -2021 -S_America -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [20]:
ls_Asia = ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia', 'China', 'Cyprus', 'Egypt',
           'Georgia', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'North Korea', 'South Korea',
           'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'Oman', 'Pakistan',
           'Philippines', 'Qatar', 'Russia', 'Saudi Arabia', 'Singapore', 'Sri Lanka', 'Syria', 'Tajikistan', 'Thailand', 'Timor-Leste',
           'Turkey', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen', 'Palestine',
           'World']  # 'Europe & Central Asia', 'South Asia', 'East Asia & Pacific', 'Middle East & North Africa'

df_Asia = df.loc[ls_Asia]   \
            .sort_values(by='total', ascending=False)
df_Asia

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Japan,84.445610,81.470000,87.570000,6.100000,81.076098,2.511707,83.587805,0.768537,84.356341,0.203659,84.560000,-0.114390,0.857805
South Korea,83.526829,80.600000,86.600000,6.000000,75.909756,5.812195,81.721951,1.504878,83.226829,0.200000,83.426829,0.100000,1.804878
Singapore,83.441463,81.100000,85.900000,4.800000,77.951220,4.543902,82.495122,1.100000,83.595122,0.870732,84.465854,-1.024390,0.946341
Israel,82.500000,80.500000,84.600000,4.100000,78.953659,3.200000,82.153659,0.651220,82.804878,-0.156098,82.648780,-0.148780,0.346341
Cyprus,81.203000,79.209000,83.204000,3.995000,76.568000,4.203000,80.771000,0.626000,81.397000,-0.006000,81.391000,-0.188000,0.432000
Maldives,79.918000,79.082000,81.019000,1.937000,70.876000,8.548000,79.424000,0.692000,80.116000,-0.241000,79.875000,0.043000,0.494000
Qatar,79.272000,78.314000,80.937000,2.623000,74.828000,4.940000,79.768000,1.222000,80.990000,-1.891000,79.099000,0.173000,-0.496000
Bahrain,78.760000,77.757000,80.018000,2.261000,75.496000,3.832000,79.328000,0.691000,80.019000,-0.845000,79.174000,-0.414000,-0.568000
Thailand,78.715000,74.518000,83.041000,8.523000,72.316000,5.099000,77.415000,1.560000,78.975000,0.299000,79.274000,-0.559000,1.300000


In [21]:
table_code_Asia = create_table(df_Asia)
output_table_code(table_code_Asia, 'Table code WBG -2021 -Asia -en.txt', destination='file')

Data has written to file


In [22]:
table_code_Asia = create_table(df_Asia, lang='ru')
output_table_code(table_code_Asia, 'Table code WBG -2021 -Asia -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [23]:
ls_Europe = ['Albania', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
             'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
             'Kazakhstan', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway',
             'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
             'Turkey', 'Ukraine', 'United Kingdom', 'Liechtenstein',
             'World', 'European Union',
             'Channel Islands', 'Faroe Islands', 'Gibraltar', 'Isle of Man']  # 'Central Europe and the Baltics', 'Europe & Central Asia']
# small countries: Andorra, Liechtenstein, Monaco, San Marino, Vatican City

df_Europe = df.loc[ls_Europe]   \
              .sort_values(by='total', ascending=False)
df_Europe

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Liechtenstein,84.402439,82.500000,86.400000,3.900000,76.826829,5.246341,82.073171,2.087805,84.160976,-2.502439,81.658537,2.743902,2.329268
Switzerland,83.851220,81.900000,85.900000,4.000000,79.680488,3.517073,83.197561,0.707317,83.904878,-0.904878,83.000000,0.851220,0.653659
Faroe Islands,83.497561,81.400000,85.700000,4.300000,78.436585,3.156098,81.592683,1.353659,82.946341,0.251220,83.197561,0.300000,1.904878
Spain,83.178049,80.300000,86.200000,5.900000,78.965854,4.263415,83.229268,0.602439,83.831707,-1.500000,82.331707,0.846341,-0.051220
Norway,83.163415,81.700000,84.700000,3.000000,78.634146,3.465854,82.100000,0.858537,82.958537,0.251220,83.209756,-0.046341,1.063415
Sweden,83.156098,81.400000,85.000000,3.600000,79.643902,2.609756,82.253659,0.856098,83.109756,-0.753659,82.356098,0.800000,0.902439
Iceland,83.117073,81.800000,84.500000,2.700000,79.653659,3.207317,82.860976,0.302439,83.163415,-0.100000,83.063415,0.053659,0.256098
Malta,82.860976,81.300000,84.500000,3.200000,78.348780,3.697561,82.046341,0.812195,82.858537,-0.509756,82.348780,0.512195,0.814634
Italy,82.795122,80.600000,85.100000,4.500000,79.778049,3.312195,83.090244,0.407317,83.497561,-1.302439,82.195122,0.600000,-0.295122


In [24]:
table_code_Europe = create_table(df_Europe)
output_table_code(table_code_Europe, 'Table code WBG -2021 -Europe -en.txt', destination='file')

Data has written to file


In [25]:
table_code_Europe = create_table(df_Europe, lang='ru')
output_table_code(table_code_Europe, 'Table code WBG -2021 -Europe -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [26]:
ls_Africa = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cabo Verde',
             "Cote d'Ivoire", 'Central African Republic', 'Chad', 'Comoros', 'DR Congo',
             'Congo, Rep.', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini',
             'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya',
             'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius',
             'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe',
             'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan',
             'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe',
             'World']  # 'Middle East & North Africa', 'Africa Eastern and Southern', 'Sub-Saharan Africa', 'Africa Western and Central'

df_Africa = df.loc[ls_Africa]   \
              .sort_values(by='total', ascending=False)
df_Africa

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Algeria,76.377000,74.879000,77.959000,3.080000,70.478000,4.632000,75.110000,1.364000,76.474000,-2.021000,74.453000,1.924000,1.267000
Cabo Verde,74.052000,69.589000,78.457000,8.868000,68.618000,6.622000,75.240000,0.764000,76.004000,-1.196000,74.808000,-0.756000,-1.188000
Morocco,74.042000,71.893000,76.398000,4.505000,66.787000,5.751000,72.538000,1.732000,74.270000,-0.350000,73.920000,0.122000,1.504000
Tunisia,73.772000,70.743000,77.145000,6.402000,73.690000,1.960000,75.650000,0.343000,75.993000,-0.701000,75.292000,-1.520000,-1.878000
Mauritius,73.680244,70.290000,77.240000,6.950000,71.663415,2.530976,74.194390,0.041463,74.235854,-0.058780,74.177073,-0.496829,-0.514146
Seychelles,73.397561,71.300000,75.600000,4.300000,72.778049,0.451220,73.229268,0.817073,74.046341,3.190244,77.236585,-3.839024,0.168293
Libya,71.911000,69.630000,74.396000,4.766000,70.680000,0.831000,71.511000,0.952000,72.463000,0.009000,72.472000,-0.561000,0.400000
World,71.327718,68.886746,73.948295,5.061549,67.700402,4.180894,71.881296,1.098420,72.979716,-0.735866,72.243850,-0.916132,-0.553578
Egypt,70.221000,67.935000,72.608000,4.673000,68.007000,2.408000,70.415000,0.943000,71.358000,-0.368000,70.990000,-0.769000,-0.194000


In [27]:
table_code_Africa = create_table(df_Africa)
output_table_code(table_code_Africa, 'Table code WBG -2021 -Africa -en.txt', destination='file')

Data has written to file


In [28]:
table_code_Africa = create_table(df_Africa, lang='ru')
output_table_code(table_code_Africa, 'Table code WBG -2021 -Africa -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [29]:
ls_Oceania = ['Australia', 'Papua New Guinea', 'New Zealand', 'Fiji', 'Solomon Islands', 'Vanuatu',
              'Samoa', 'Kiribati', 'Micronesia', 'Tonga', 'Marshall Islands', 'Nauru', 'Tuvalu',
              'French Polynesia', 'New Caledonia', 'Guam', 'Pacific island small states',
              'World']
# Database does not contain data for 'Palau', 'American Samoa', 'Northern Mariana Islands'

df_Oceania = df.loc[ls_Oceania]   \
                 .sort_values(by='total', ascending=False)
df_Oceania

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Australia,83.300000,81.300000,85.400000,4.100000,79.234146,3.065854,82.300000,0.600000,82.900000,0.300000,83.200000,0.100000,1.000000
New Zealand,82.207317,80.500000,84.000000,3.500000,78.636585,2.820244,81.456829,0.599268,82.056098,0.200000,82.256098,-0.048780,0.750488
French Polynesia,79.486000,76.975000,82.632000,5.657000,79.536000,2.678000,82.214000,0.782000,82.996000,-0.779000,82.217000,-2.731000,-2.728000
New Caledonia,79.127341,75.361000,83.082000,7.721000,75.163415,2.073171,77.236585,3.183122,80.419707,0.415244,80.834951,-1.707610,1.890756
Guam,76.656000,72.982000,80.704000,7.722000,74.417000,2.332000,76.749000,0.968000,77.717000,-1.105000,76.612000,0.044000,-0.093000
Samoa,72.767000,70.312000,75.503000,5.191000,70.746000,1.717000,72.463000,-0.306000,72.157000,0.611000,72.768000,-0.001000,0.304000
World,71.327718,68.886746,73.948295,5.061549,67.700402,4.180894,71.881296,1.098420,72.979716,-0.735866,72.243850,-0.916132,-0.553578
Tonga,70.986000,68.416000,73.719000,5.303000,68.978000,1.562000,70.540000,0.331000,70.871000,0.057000,70.928000,0.058000,0.446000
Micronesia,70.710000,67.120000,74.608000,7.488000,69.769000,1.760000,71.529000,-0.452000,71.077000,-0.403000,70.674000,0.036000,-0.819000


In [30]:
table_code_Oceania = create_table(df_Oceania)
output_table_code(table_code_Oceania, 'Table code WBG -2021 -Oceania -en.txt', destination='file')

Data has written to file


In [31]:
table_code_Oceania = create_table(df_Oceania, lang='ru')
output_table_code(table_code_Oceania, 'Table code WBG -2021 -Oceania -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [32]:
ls_CIS = ['Armenia', 'Azerbaijan', 'Belarus', 'Kazakhstan', 'Kyrgyzstan', 'Moldova',
          'Russia', 'Tajikistan', 'Uzbekistan', 'Turkmenistan']  # 'Georgia', 'Ukraine'

df_CIS = df.loc[ls_CIS]   \
           .sort_values(by='total', ascending=False)
df_CIS

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
Belarus,72.370683,67.298,77.697,10.399,68.912195,4.058537,72.970732,1.256098,74.226829,-1.769610,72.45722,-0.086537,-0.600049
Armenia,72.043000,66.554,77.350,10.796,70.624000,3.434000,74.058000,1.381000,75.439000,-3.266000,72.17300,-0.130000,-2.015000
Kyrgyzstan,71.900000,67.900,76.100,8.200,68.558537,1.843902,70.402439,1.197561,71.600000,0.200000,71.80000,0.100000,1.497561
Tajikistan,71.594000,69.568,73.734,4.166,63.260000,5.809000,69.069000,1.798000,70.867000,-2.873000,67.99400,3.600000,2.525000
Uzbekistan,70.862000,68.334,73.390,5.056,65.724000,4.510000,70.234000,1.110000,71.344000,-1.013000,70.33100,0.531000,0.628000
Kazakhstan,70.230000,66.330,74.030,7.700,65.450000,5.990000,71.440000,1.740000,73.180000,-1.810000,71.37000,-1.140000,-1.210000
Azerbaijan,69.366000,65.646,73.287,7.641,64.891000,6.225000,71.116000,1.986000,73.102000,-6.234000,66.86800,2.498000,-1.750000
Russia,69.361220,64.210,74.770,10.560,65.483659,5.260000,70.743659,2.340244,73.083902,-1.745122,71.33878,-1.977561,-1.382439
Turkmenistan,69.264000,65.865,72.663,6.798,65.026000,3.588000,68.614000,0.388000,69.002000,-0.315000,68.68700,0.577000,0.650000


In [33]:
table_code_CIS = create_table(df_CIS)
output_table_code(table_code_CIS, 'Table code WBG -2021 -CIS -en.txt', destination='file')

Data has written to file


In [34]:
table_code_CIS = create_table(df_CIS, lang='ru')
output_table_code(table_code_CIS, 'Table code WBG -2021 -CIS -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [35]:
ls_regions = [
    'European Union', 'Central Europe and the Baltics',
    'Europe & Central Asia',
    'South Asia',
    'East Asia & Pacific',
    'Middle East & North Africa',
    'Africa Eastern and Southern', 'Sub-Saharan Africa', 'Africa Western and Central',
    'North America', 'Latin America & Caribbean',
    'Arab World'
]

# exclude countries with population <50,000
ls_dropped_territories = ['St. Kitts and Nevis', 'Turks and Caicos Islands', 'Sint Maarten', 'Gibraltar',
                          'Collectivity of Saint Martin', 'British Virgin Islands', 'Tuvalu', 'Nauru']  # 'Palau'

In [36]:
df_all_countries = df.drop(ls_regions + ls_dropped_territories)
print(len(df_all_countries))

201


In [37]:
table_code_all_countries = create_table(df_all_countries)
output_table_code(table_code_all_countries, 'Table code WBG -2021 -all_countries -en.txt', destination='file')

Data has written to file


In [38]:
table_code_all_countries = create_table(df_all_countries, lang='ru')
output_table_code(table_code_all_countries, 'Table code WBG -2021 -all_countries -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [39]:
# create code for placing info in Wikipedia
def create_table_for_regions(df, lang='en'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    def chval(x, prec=2):  # change_value
        return 'style="background:#fffae0;"| —' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"| {x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;"| −{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"| −{x:0.{prec}f}'  # normal sorting is required '−' before 0.0 - bug in wiki
    
    def chval_bold(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"| \'\'\'{x:0.{prec}f}\'\'\''

    if lang=='ru':
        file_header='wbg_stats_header_2021_ru.txt'
        ptn_1 = 'флагификация'
        ptn_2 = 'флаг'
        prettify_name = {
            'European Union': '[[Европейский союз]]',
            'Central Europe and the Baltics': '[[Центральная Европа]] и [[Прибалтика]]',
            'Europe & Central Asia': '[[Европа]] и [[Центральная Азия]]',
            'South Asia': '[[Южная Азия]]',
            'East Asia & Pacific': '[[Восточная Азия]] и [[Австралия и Океания|Океания]]',
            'Middle East & North Africa': '[[Страны MENA|Ближний Восток и Северная Африка]]',
            'Africa Eastern and Southern': '[[Восточная Африка|Восточная]] и [[Южная Африка|Южная]] Африка',
            'Sub-Saharan Africa': '[[Чёрная Африка|Африка к югу от Сахары]]',
            'Africa Western and Central': '[[Западная Африка|Западная]] и [[Центральная Африка|Центральная]] Африка',
            'North America': '[[Северная Америка]]',
            'Latin America & Caribbean': '[[Латинская Америка]] и [[Антильские острова|Карибы]]',
            'Arab World': 'Арабский мир',
            'World': 'Мир'
        }
    else:
        file_header='wbg_stats_header_2021_en.txt'
        ptn_1 = 'flaglist'
        ptn_2 = 'flagicon'
        prettify_name = {
            'European Union': '[[European Union]]',
            'Central Europe and the Baltics': '[[Central Europe]] & the [[Baltic states|Baltics]]',
            'Europe & Central Asia': '[[Europe]] & [[Central Asia]]',
            'South Asia': '[[South Asia]]',
            'East Asia & Pacific': '[[Asia-Pacific|East Asia & Pacific]]',
            'Middle East & North Africa': '[[MENA|Middle East & North Africa]]',
            'Africa Eastern and Southern': 'Africa [[East Africa|Eastern]] & [[Southern Africa|Southern]]',
            'Sub-Saharan Africa': '[[Sub-Saharan Africa]]',
            'Africa Western and Central': 'Africa [[West Africa|Western]] & [[Central Africa|Central]]',
            'North America': '[[North America]]',
            'Latin America & Caribbean': '[[Latin America and the Caribbean|Latin America & Caribbean]]'
        }
        
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        st_header = fh.read()
        
    st_header = st_header.strip()
    st = ''
        
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name in ['World']:
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|align=center| \'\'\'{prettify_name.get(ser.name, ser.name)}\'\'\' ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#eaf3ff;"| \'\'\'{if_value(ser.male, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#fee7f6;"| \'\'\'{if_value(ser.female, 2)}\'\'\' ' + \
                  f'||style="text-align:center; background:#fff8dc;"| \'\'\'{if_value(ser.gender_gap, 2)}\'\'\' ' + \
                  f'||style="text-align:center; border-left-width:2px;"| \'\'\'{if_value(ser["2014_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2019_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| \'\'\'{if_value(ser["2020_t"], 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| \'\'\'{if_value(ser.total, 2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2014→2021"], 2)} '
        else:
            st += '\n' + '|-\n' + \
                  f'|align=left|{prettify_name.get(ser.name, ser.name)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||style="text-align:center; background:#eaf3ff;"| {if_value(ser.male, 2)} ' + \
                  f'||style="text-align:center; background:#fee7f6;"| {if_value(ser.female, 2)} ' + \
                  f'||style="text-align:center; background:#fff8dc;"| {if_value(ser.gender_gap, 2)} ' + \
                  f'||style="text-align:center; border-left-width:2px;"| {if_value(ser["2014_t"], 2)} ' + \
                  f'||{chval(ser["2014→2019"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2019_t"], 2)} ' + \
                  f'||{chval(ser["2019→2020"], 2)} ' + \
                  f'||style="text-align:center;"| {if_value(ser["2020_t"], 2)} ' + \
                  f'||{chval(ser["2020→2021"], 2)} ' + \
                  f'||style="text-align:center; background:#e0ffd8;"| {if_value(ser.total, 2)} ' + \
                  f'||{chval(ser["2014→2021"], 2)}'
    st += '\n|}'

    # WARNING: be sure that inline styles do not contains numbers with comma
    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits
        
    st = st_header + st

    return st

In [40]:
df_regions = df.loc[ls_regions + ['World']]   \
               .sort_values(by='total', ascending=False)
df_regions

,total,male,female,gender_gap,2000_t,2000→2014,2014_t,2014→2019,2019_t,2019→2020,2020_t,2020→2021,2014→2021
,,,,,,,,,,,,,
European Union,80.387131,77.658520,83.265708,5.607188,77.075317,3.796666,80.871982,0.442876,81.314858,-0.862765,80.452093,-0.064962,-0.484852
North America,76.977263,74.238122,79.853964,5.615842,76.885154,2.251130,79.136284,0.005428,79.141712,-1.677840,77.463872,-0.486610,-2.159022
Europe & Central Asia,76.631526,73.511858,79.951636,6.439778,73.132090,4.164361,77.296451,0.889797,78.186248,-1.227058,76.959190,-0.327664,-0.664925
East Asia & Pacific,76.393934,73.639441,79.329139,5.689697,71.552570,4.178735,75.731305,1.066470,76.797775,-0.035835,76.761940,-0.368005,0.662629
Central Europe and the Baltics,74.932414,71.323053,78.730265,7.407212,72.719725,3.955899,76.675624,0.591221,77.266845,-1.179128,76.087717,-1.155303,-1.743210
Middle East & North Africa,72.745907,70.640188,75.084930,4.444742,69.282178,3.341685,72.623863,1.335252,73.959115,-1.009347,72.949768,-0.203862,0.122043
Latin America & Caribbean,72.147446,68.802793,75.699847,6.897054,71.228436,3.322211,74.550648,0.487702,75.038349,-1.999875,73.038474,-0.891028,-2.403202
World,71.327718,68.886746,73.948295,5.061549,67.700402,4.180894,71.881296,1.098420,72.979716,-0.735866,72.243850,-0.916132,-0.553578
Arab World,70.814483,68.832433,72.989694,4.157261,67.297331,3.195298,70.492629,1.351997,71.844626,-0.921266,70.923360,-0.108877,0.321854


In [41]:
table_code_regions = create_table_for_regions(df_regions)
output_table_code(table_code_regions, 'Table code WBG -2021 -z_regions -en.txt', destination='file')

Data has written to file


In [42]:
table_code_regions = create_table_for_regions(df_regions, lang='ru')
output_table_code(table_code_regions, 'Table code WBG -2021 -z_regions -ru.txt', destination='file')

Data has written to file


<br />
<br />
<br />

In [43]:
# check that all countries are considered for local tables
ls_all_countries = sorted(df.index.to_list())
print(len(ls_all_countries))

221


In [44]:
# combine all local lists into one

import itertools
all_local_lists = sorted(set(itertools.chain(ls_N_America, ls_S_America, ls_Asia, ls_Europe, ls_Africa, ls_Oceania, ls_regions)))
print(len(all_local_lists))

221


In [45]:
[country for country in ls_all_countries if country not in all_local_lists]

[]

In [46]:
[country for country in all_local_lists if country not in ls_all_countries]

[]